In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import random

# 후보 리스트를 담을 변수
candidates = []
selected_images = []

# 대회 정보 저장 변수
tournaments = {}

# 이미지 업로드 함수
def upload_images():
    file_paths = filedialog.askopenfilenames(title="여러 이미지 파일 선택", filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    if file_paths:
        for file_path in file_paths:
            image = Image.open(file_path)
            image.thumbnail((200, 200))  # 이미지 크기 조정
            image_tk = ImageTk.PhotoImage(image)
            selected_images.append({'image': image_tk, 'image_path': file_path, 'name': ""})
        update_selected_images()

# 선택된 이미지 목록 업데이트
def update_selected_images():
    for widget in scrollable_frame.winfo_children():
        widget.destroy()

    for image_data in selected_images:
        frame = tk.Frame(scrollable_frame)
        frame.pack(pady=10)

        label_image = tk.Label(frame, image=image_data['image'])
        label_image.image = image_data['image']
        label_image.pack(side=tk.LEFT)

        name_entry = tk.Entry(frame, font=("Arial", 12))
        name_entry.pack(side=tk.LEFT, padx=10)
        image_data['name_entry'] = name_entry

    canvas.config(scrollregion=canvas.bbox("all"))

# 후보 저장 함수
def save_candidate():
    global candidates
    for image_data in selected_images:
        name = image_data['name_entry'].get()
        if name:
            candidates.append({'name': name, 'image': image_data['image'], 'image_path': image_data['image_path']})
            saved_candidates_listbox.insert(tk.END, f"{len(candidates)}. {name}")
        else:
            messagebox.showwarning("입력 오류", "모든 후보에 대해 이름을 입력해주세요.")
            return

    selected_images.clear()
    update_selected_images()

# 대회 저장 함수
def save_tournament():
    tournament_name = title_entry.get()
    if not tournament_name:
        messagebox.showwarning("입력 오류", "대회 타이틀을 입력해주세요.")
        return

    if not candidates:
        messagebox.showwarning("저장 오류", "저장할 후보가 없습니다.")
        return
    
    saved_list = [f"{candidate['name']} (이미지: {candidate['image_path']})" for candidate in candidates]
    tournament_info = f"{tournament_name}:\n" + "\n".join(saved_list)
    
    saved_photos_listbox.insert(tk.END, tournament_info)
    tournaments[tournament_name] = candidates[:]  # 대회 정보를 저장
    messagebox.showinfo("저장 완료", f"{tournament_name} 대회가 저장되었습니다.")

# 대회 선택 후 시작 버튼
def start_selected_tournament():
    selected_index = saved_photos_listbox.curselection()
    if not selected_index:
        messagebox.showwarning("선택 오류", "대회를 선택해주세요.")
        return

    tournament_name = saved_photos_listbox.get(selected_index)
    candidates.clear()
    candidates.extend(tournaments[tournament_name.split(":")[0]])  # 선택한 대회의 후보들을 가져옴

    # 새로운 대회 화면 열기
    open_tournament_window()

# 새로운 대회 화면 열기
def open_tournament_window():
    tournament_window = tk.Toplevel(window)
    tournament_window.title("대회 진행")
    tournament_window.geometry("800x600")

    # 대회 진행 라벨
    tournament_label = tk.Label(tournament_window, text="대회 시작!", font=("Arial", 20))
    tournament_label.pack(pady=20)

    # 후보 이미지 라벨
    candidate1_label = tk.Label(tournament_window, text="", font=("Arial", 14), compound="top")
    candidate1_label.pack(pady=10)

    candidate2_label = tk.Label(tournament_window, text="", font=("Arial", 14), compound="top")
    candidate2_label.pack(pady=10)

    # 결과 표시 라벨
    result_label = tk.Label(tournament_window, text="", font=("Arial", 14))
    result_label.pack(pady=10)

    # 승자 선택 버튼
    def select_winner(winner):
        nonlocal current_round_candidates
        if len(current_round_candidates) > 1:
            next_round_candidates = current_round_candidates[::2] if winner == 1 else current_round_candidates[1::2]
            current_round_candidates = next_round_candidates
            run_round()
        else:
            result_label.config(text=f"최종 승자: {current_round_candidates[0]['name']}")

    # 현재 라운드 후보들
    current_round_candidates = candidates[:]
    
    # 라운드 진행 함수
    def run_round():
        if len(current_round_candidates) < 2:
            result_label.config(text="후보가 부족합니다.")
            return
        
        random.shuffle(current_round_candidates)
        candidate1 = current_round_candidates[0]
        candidate2 = current_round_candidates[1]

        candidate1_label.config(image=candidate1['image'], text=candidate1['name'])
        candidate2_label.config(image=candidate2['image'], text=candidate2['name'])

        button1 = tk.Button(tournament_window, text=f"{candidate1['name']} 승자", command=lambda: select_winner(1))
        button2 = tk.Button(tournament_window, text=f"{candidate2['name']} 승자", command=lambda: select_winner(2))

        button1.pack(pady=5)
        button2.pack(pady=5)

    # 첫 라운드 시작
    run_round()

# GUI 설정
window = tk.Tk()
window.title("이상형 월드컵 소프트웨어")
window.geometry("1200x800")

# 화면을 3개 구역으로 나누기 위한 PanedWindow 위젯 사용
paned_window = tk.PanedWindow(window, orient="horizontal")
paned_window.pack(fill=tk.BOTH, expand=True)

# 왼쪽 구역
left_frame = tk.Frame(paned_window, bg="lightgray")
paned_window.add(left_frame)
paned_window.paneconfig(left_frame, minsize=400)

# 중앙 구역
center_frame = tk.Frame(paned_window)
paned_window.add(center_frame)
paned_window.paneconfig(center_frame, minsize=400)

# 오른쪽 구역
right_frame = tk.Frame(paned_window)
paned_window.add(right_frame)
paned_window.paneconfig(right_frame, minsize=400)

# 기준선 추가
def add_separator():
    separator_left = tk.Canvas(window, width=2, bg='#A9A9A9')
    separator_left.place(x=400, relheight=1)
    separator_center = tk.Canvas(window, width=2, bg='#A9A9A9')
    separator_center.place(x=800, relheight=1)

add_separator()

# 왼쪽 구역 상단: 저장된 이상형 월드컵 목록
saved_label = tk.Label(left_frame, text="저장된 이상형 월드컵 목록", font=("Arial", 14))
saved_label.pack(pady=10)

saved_photos_listbox = tk.Listbox(left_frame, height=15, width=30, font=("Arial", 12))
saved_photos_listbox.pack(padx=20, pady=10)

# 왼쪽 구역 하단: 저장된 후보 목록
saved_candidates_label = tk.Label(left_frame, text="저장된 후보 목록", font=("Arial", 14))
saved_candidates_label.pack(pady=10)

saved_candidates_listbox = tk.Listbox(left_frame, height=15, width=30, font=("Arial", 12))
saved_candidates_listbox.pack(padx=20, pady=10)

# 중앙 구역
title_label = tk.Label(center_frame, text="대회 타이틀 입력", font=("Arial", 14))
title_label.pack(pady=10)

title_entry = tk.Entry(center_frame, font=("Arial", 12))
title_entry.pack(pady=5)

# 후보 이미지 추가
add_button = tk.Button(center_frame, text="여러 후보 이미지 추가", command=upload_images, font=("Arial", 14))
add_button.pack(pady=10)

# 라운드 선택 (라디오버튼)
round_label = tk.Label(center_frame, text="라운드 선택", font=("Arial", 12))
round_label.pack(pady=5)

round_var = tk.StringVar(value="8")
round_4_button = tk.Radiobutton(center_frame, text="4강", variable=round_var, value="4")
round_4_button.pack(anchor="w")
round_8_button = tk.Radiobutton(center_frame, text="8강", variable=round_var, value="8")
round_8_button.pack(anchor="w")
round_16_button = tk.Radiobutton(center_frame, text="16강", variable=round_var, value="16")
round_16_button.pack(anchor="w")
round_32_button = tk.Radiobutton(center_frame, text="32강", variable=round_var, value="32")
round_32_button.pack(anchor="w")
round_64_button = tk.Radiobutton(center_frame, text="64강", variable=round_var, value="64")
round_64_button.pack(anchor="w")

# 대회 시작 버튼
start_button = tk.Button(center_frame, text="대회 시작", command=start_tournament, font=("Arial", 14))
start_button.pack(pady=10)

# 대회 저장 버튼
save_button = tk.Button(center_frame, text="대회 저장", command=save_tournament, font=("Arial", 14))
save_button.pack(pady=10)

# 대회 선택 후 시작 버튼
select_start_button = tk.Button(center_frame, text="선택된 대회 시작", command=start_selected_tournament, font=("Arial", 14))
select_start_button.pack(pady=10)

# 오른쪽 구역: 선택된 후보 이미지들 및 이름 입력
right_label = tk.Label(right_frame, text="선택된 후보", font=("Arial", 14))
right_label.grid(row=0, column=0, pady=10)

# 오른쪽 구역의 캔버스 및 스크롤바 설정
canvas = tk.Canvas(right_frame)
scrollbar = tk.Scrollbar(right_frame, orient="vertical", command=canvas.yview, width=20)  # 스크롤바 폭을 20으로 설정
scrollable_frame = tk.Frame(canvas)

scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)

canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.config(yscrollcommand=scrollbar.set)

# 캔버스와 스크롤바 배치
canvas.grid(row=1, column=0, sticky="nsew")  # grid로 배치하여 캔버스 크기 조정
scrollbar.grid(row=1, column=1, sticky="ns")  # 스크롤바는 오른쪽에 배치

# 마우스 휠로 스크롤할 수 있도록 이벤트 추가
def on_mouse_wheel(event):
    canvas.yview_scroll(-1 * (event.delta // 120), "units")

# 마우스 휠 이벤트 바인딩
canvas.bind_all("<MouseWheel>", on_mouse_wheel)

# 후보 저장 버튼을 화면 하단에 배치
save_candidate_button = tk.Button(right_frame, text="후보 저장", command=save_candidate, font=("Arial", 14))
save_candidate_button.grid(row=2, column=0, pady=10, sticky="nsew")  # grid로 아래쪽에 배치

# 오른쪽 프레임의 크기 확장
right_frame.grid_rowconfigure(0, weight=0)
right_frame.grid_rowconfigure(1, weight=1)
right_frame.grid_rowconfigure(2, weight=0)  # 버튼이 화면 아래에 배치되도록 설정

# 대회 시작 시 결과 표시 라벨 추가
result_label = tk.Label(window, text="", font=("Arial", 14))
result_label.pack(pady=10)

# 후보 1과 후보 2 라벨 추가
candidate1_label = tk.Label(window, text="", font=("Arial", 14), compound="top")
candidate1_label.pack(pady=10)

candidate2_label = tk.Label(window, text="", font=("Arial", 14), compound="top")
candidate2_label.pack(pady=10)

# 메인 루프 실행
window.mainloop()
